Test code generation with models of various sophistication following on from release of o1 and seeing strategies for generating plans and code using the reportoire of model stengths and weaknesses

with gpt-4o-mini

Write a python library of functions with a demonstration of it's usage to calculate APR from investing in shares

1. A share can be bought and sold within a day or over a longer period, months or years.

2. When a share is bought, the date and time is captured to the minute.

3. When a share is sold, the date and time is captured to the minute.

4. The application should be able to calculate APR based on a combination of sold shares
(realised profit/loss) together with the current value of shares currently held within a portfolio  

5. The application should report the realised profit/loss of shares no longer owned,
together with its bought and sold datetimes and the days elapsed between purchase and sale.
Additionally the effective APR of the share sale should be reported.

6. The application should report the unrealised profit/loss of shares still held, together
with its bought date and time. Additionally the effective APR of the share sale should be reported.

7. Within the application, all date and times should be human readable

8. The demonstation of usage should give examples of realised share dealings held for 1 day, 5 days and 6 months respectively

9. The demonstation of usage should give two examples of unrealised share purchases, one is a profit and one is a loss

10. The details of realised and unrealised dealing should be pretty printed


Respond with the code only! Nothing else!

In [ ]:

from datetime import datetime
from collections import namedtuple

class Share:
    def __init__(self, symbol, purchase_price, quantity, purchase_date):
        self.symbol = symbol
        self.purchase_price = purchase_price
        self.quantity = quantity
        self.purchase_date = purchase_date
        self.sales = []

    def sell(self, sell_price, quantity, sell_date):
        self.sales.append((sell_price, quantity, sell_date))

    def realised_profit_loss(self):
        total_spent = self.purchase_price * self.quantity
        total_earned = sum(price * qty for price, qty, _ in self.sales)
        return total_earned - total_spent

    def unrealised_profit_loss(self, current_price):
        return (current_price - self.purchase_price) * self.quantity

    def days_held(self, sell_date):
        return (sell_date - self.purchase_date).days

    def effective_apr(self, sell_price, quantity, sell_date):
        profit = (sell_price - self.purchase_price) * quantity
        days = self.days_held(sell_date)
        apr = (profit / (self.purchase_price * quantity)) * (365 / days) * 100 if days > 0 else 0
        return apr

    def display_realised(self):
        for sell_price, quantity, sell_date in self.sales:
            days = self.days_held(sell_date)
            effective_apr = self.effective_apr(sell_price, quantity, sell_date)
            print(f"Realised Sale - Symbol: {self.symbol}, Sold Price: {sell_price}, "
                  f"Quantity: {quantity}, Date Sold: {sell_date}, Days Held: {days}, "
                  f"Profit/Loss: {self.realised_profit_loss()}, Effective APR: {effective_apr:.2f}%")

    def display_unrealised(self, current_price):
        unrealised_pl = self.unrealised_profit_loss(current_price)
        print(f"Unrealised Holding - Symbol: {self.symbol}, Current Price: {current_price}, "
              f"Bought Price: {self.purchase_price}, Quantity: {self.quantity}, "
              f"Unrealised Profit/Loss: {unrealised_pl}, "
              f"Effective APR: N/A (not sold)")


def demo_usage():
    # Realised Positions
    share1 = Share('AAPL', 150.00, 10, datetime(2023, 1, 1, 12, 0))
    share1.sell(155.00, 5, datetime(2023, 1, 2, 12, 0))  # Sold 5 shares after 1 day
    share1.sell(160.00, 5, datetime(2023, 1, 6, 12, 0))  # Sold 5 shares after 5 days
    share1.display_realised()

    share2 = Share('TSLA', 600.00, 10, datetime(2023, 6, 1, 12, 0))
    share2.sell(650.00, 10, datetime(2023, 12, 1, 12, 0))  # Sold after 6 months
    share2.display_realised()

    # Unrealised Positions
    share3 = Share('GOOGL', 2500.00, 2, datetime(2023, 7, 1, 12, 0))
    share3.display_unrealised(2600.00)  # Currently valued at 2600

    share4 = Share('AMZN', 3300.00, 2, datetime(2023, 8, 1, 12, 0))
    share4.display_unrealised(3200.00)  # Currently valued at 3200


# Run the demonstration
demo_usage()


Realised Sale - Symbol: AAPL, Sold Price: 155.0, Quantity: 5, Date Sold: 2023-01-02 12:00:00, Days Held: 1, Profit/Loss: 75.0, Effective APR: 1216.67%
Realised Sale - Symbol: AAPL, Sold Price: 160.0, Quantity: 5, Date Sold: 2023-01-06 12:00:00, Days Held: 5, Profit/Loss: 75.0, Effective APR: 486.67%
Realised Sale - Symbol: TSLA, Sold Price: 650.0, Quantity: 10, Date Sold: 2023-12-01 12:00:00, Days Held: 183, Profit/Loss: 500.0, Effective APR: 16.62%
Unrealised Holding - Symbol: GOOGL, Current Price: 2600.0, Bought Price: 2500.0, Quantity: 2, Unrealised Profit/Loss: 200.0, Effective APR: N/A (not sold)
Unrealised Holding - Symbol: AMZN, Current Price: 3200.0, Bought Price: 3300.0, Quantity: 2, Unrealised Profit/Loss: -200.0, Effective APR: N/A (not sold)


from gemini - code could be merged

In [ ]:
# prompt: write a python library to get current share price from a ticker symbol and give an example

import yfinance as yf

def get_current_share_price(ticker_symbol):
    """
    Retrieves the current share price for a given ticker symbol using yfinance.

    Args:
        ticker_symbol: The stock ticker symbol (e.g., "AAPL", "MSFT").

    Returns:
        The current share price as a float, or None if the ticker symbol is invalid or data is unavailable.
    """
    try:
        ticker_data = yf.Ticker(ticker_symbol)
        current_price = ticker_data.info.get('currentPrice')
        if current_price is not None:
            return current_price
        else:
            return None  # Or raise an exception if you prefer
    except Exception as e:
        print(f"Error retrieving data for {ticker_symbol}: {e}")
        return None


# Example usage
if __name__ == "__main__":
    ticker = "AAPL"  # Replace with your desired ticker symbol
    price = get_current_share_price(ticker)

    if price:
        print(f"The current price of {ticker} is: ${price:.2f}")
    else:
        print(f"Could not retrieve current price for {ticker}")

The current price of AAPL is: $253.01
